In [3]:
from pymongo import MongoClient
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pprint
import time
import json
from contextlib import contextmanager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re

In [4]:
client = MongoClient()
capstone_db=client['capstone']
rating = capstone_db['femi']

In [5]:
url_main='https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=1&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release'

In [6]:
def get_urls(x):
    driver = webdriver.Chrome(executable_path='./src/chromedriver')
    driver.get(x)
    time.sleep(15)
    show_more_button = driver.find_element_by_class_name("btn.btn-secondary-rt.mb-load-btn")
    
    while(True):
           
        try:
            show_more_button.click()
            time.sleep(2)
            
        except ElementNotVisibleException:
            break
        except TimeoutException:
            break
        except StaleElementReferenceException:
            break
        
        
    #record each movie title and its url inside dict
    soup = BeautifulSoup(driver.page_source,"lxml")
    links=[]
    for link in soup.findAll('a', attrs={'href': re.compile("/m/")}):
        links.append(link.get('href'))
    links2=[]
    for link in links:
        if link not in links2:
            links2.append(link)
    #movies = soup.find_all('h3', {'class' :"movieTitle"})
    #movies= [movie.text for movie in movies]
    return links2
    driver.quit()

In [27]:
def get_info_from_urls(x):
    p=0
    for url in x:
        
        r = requests.get(url)
        if r.status_code ==200:
            soup = BeautifulSoup(r.text, 'html.parser')

            movie = soup.find('h1', class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").text
            rating_tags = soup.find_all('span', class_="mop-ratings-wrap__percentage")
            if len(rating_tags)==2:
                tomatometer,audience = [i for i in rating_tags]
                tomatometer = tomatometer.text.strip('\n').strip().strip('%')
                audience = audience.text.strip('\n').strip().strip('%')
                date = soup.find('time').text
                rating.insert_one({'movie':movie, 'tomatometer':tomatometer, 'audience':audience, 'date':date})
                p+=1
                print(f"Added {movie} to Database ({(p/len(x))*100}% done.)")
            
            else: 
                continue
        else:
            continue

        time.sleep(10)

In [8]:
urls = get_urls(url_main)

In [9]:
def get_full_urls(x):
    full_urls = []
    first_half = 'http://www.rottentomatoes.com'
    for url in x:
        full_urls.append(first_half+url)
    return full_urls
    

In [10]:
full_urls= get_full_urls(urls)

In [21]:
full_urls[300:302]

['http://www.rottentomatoes.com/m/hellraiser_hellseeker',
 'http://www.rottentomatoes.com/m/dracula_ii_ascension']

In [28]:
get_info_from_urls(full_urls[316:318])

Added The Toolbox Murders to Database (50.0% done.)
Added Killing Me Softly to Database (100.0% done.)
